# ASTR302 Lab 3: The Basics of Starting an Observational Program

In this Lab you will learn how to explore catalogs to define a sample that can be observed on a given night. This is an easy one - you just need to try to understand how things work. There are no specific questions for you to answer (so you don't need to email me this one).

## Using DATALAB

We have been focusing on statistics, so we are going to take a break - even though there is plenty more to discuss - and do some astronomy. We will begin to use the DATALAB environment at NOIRLab to make finder charts of our targets. Finder charts are helpful in confirming that you are observing the correct source.




## Selecting a sample of galaxies

Planning for any observing program starts with the selection of targets. Of course, this is influenced by the scientific questions one is asking - but for our purpose we are going to skip the motivation and just get to the target selection. In this case our targets are going to be galaxies that satisfy a set of criteris that we will specify.

Before we start, lets import the packages you will be needing for this Lab.


In [1]:
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
from scipy.optimize import curve_fit
from astropy.io import fits
from astropy.table import Table
import astropy.units as u
from astropy.coordinates import SkyCoord

import requests
import pandas as pd
from PIL import Image, ImageDraw, ImageFilter
import time
import os

!pip install --upgrade pip
!pip install astroplan
!pip install pytest-astropy --upgrade
!pip install pytz
!pip install jinja2
!pip install dustmaps
!pip install ctapipe

from dustmaps.sfd import SFDQuery
from dustmaps.config import config
config['data_dir'] = '.'
import dustmaps.sfd
dustmaps.sfd.fetch()
    
from ctapipe.utils import get_bright_stars

from astroplan import Observer, FixedTarget
from astropy.time import Time
from astroplan.plots import plot_airmass

from astroplan import observability_table
from astroplan import (AltitudeConstraint, AirmassConstraint, AtNightConstraint)


Defaulting to user installation because normal site-packages is not writeable
DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
Defaulting to user installation because normal site-packages is not writeable
DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
Defaulting to user installation because normal site-packages is not writeable
DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number.

Now, we going to start with a catalog of large (as seen on the sky) galaxies taken from images obtained for the Sloan and Legacy Survey. The catalog is called the Siena Galaxy Catalog (2020) or SGA-2020.fits. Go ahead and download it and put it in your DATALAB directory. We will then load it in and convert to a Pandas dataframe.

In [2]:
# siena_2020 galaxy catalog (download from website SGA-2020.fits) - need to change path to your version
#hdul = fits.open('SGA-2020.fits',memmap=True)
#galaxy_data = Table(hdul[1].data)
#hdul.close()

galaxy_data = pd.read_csv('SGA_2020_light.csv')

# convert to a pandas dataframe structure
#galaxy_df = galaxy_data.to_pandas()

In [3]:
# here we set the parameters of interest
print('There are',len(galaxy_data),'targets in ATLAS')    

# parameter cuts for galaxy sample

lower_dec = 20        # lower limit declination
lower_diam = 4        # lower limit diameter size (arcmin)
upper_diam = 8        # upper limit diameter size (arcmin)
BA_max = 0.8          # upper limit on B/A (avoid face-on galaxies)
bright_star_rad = 1   # search radius for bright stars (degrees)
max_ebv = 0.04        # maximum allowed value of E(B-V) to limit Galactic extinction
max_redshift = 0.005  # z, limit corresponds to ~ 21 Mpc to focus on nearby galaxies

#select all galaxies in correct coordinate range, size range, and inclination
galaxy_targets = galaxy_data[(galaxy_data['DEC'] > lower_dec) & (galaxy_data['D26'] > lower_diam) & (galaxy_data['D26'] < upper_diam) & (galaxy_data['BA'] < BA_max) & (galaxy_data['Z_LEDA']<max_redshift)]
print(len(galaxy_targets),'satisfy RA/Dec, size, b/a, and redshift criteria')

There are 383620 targets in ATLAS
138 satisfy RA/Dec, size, b/a, and redshift criteria


Now we move on to estimating the Galactic extinction at the location of each galaxy. For this we using the Schlegel, Finkbeiner and Davis (sfd) maps that are available as part of the SFDQuery call - don't you just love that there is all this infrastructure already available!

In [4]:
# get extinction at each location  (see https://dustmaps.readthedocs.io/en/latest/maps.html for range of options)

coords = SkyCoord(galaxy_targets['RA']*u.deg, galaxy_targets['DEC']*u.deg, frame='icrs')
sfd = SFDQuery()
galaxy_targets['ebv'] = sfd(coords)

#apply selection criteria
galaxy_targets = galaxy_targets[(galaxy_targets['ebv'] < max_ebv)]

print('Of those',len(galaxy_targets),'satisfy dust criterion')

Of those 115 satisfy dust criterion


<ipython-input-4-3201b5b098ca>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  galaxy_targets['ebv'] = sfd(coords)


Finally, we are going to search for nearby bright stars that could cause us problems in our observations.

In [5]:
# get brights stars at each remaining location and flag those with bright stars
# code will automatically download Yale Bright Star Catalog if you don't already have it

bright_flag=[]

for j in range(0,len(galaxy_targets)):
    br_stars = get_bright_stars(coords[j],bright_star_rad*u.deg)
    
    if (len(br_stars)>0):
        bright_flag = np.append(bright_flag,1)
    else:
        bright_flag = np.append(bright_flag,0)

galaxy_targets_new = galaxy_targets[(bright_flag<1)]
galaxy_targets_new.reset_index(inplace = True)

print(' ')
print('Of those',len(galaxy_targets_new),'do not have a Yale Bright Star Catalog star within',bright_star_rad,'degree(s)')

# now we will just write to a text file the targets that satisfy the criteria
target_file = open('galaxy_targets.txt', 'w')

for j in range(0,len(galaxy_targets_new)):
    print(galaxy_targets_new['GALAXY'][j] , galaxy_targets_new['RA'][j], galaxy_targets_new['DEC'][j])

#    print("%9s %10.6f %10.6f" % (galaxy_targets_new['GALAXY'][j], galaxy_targets_new['RA'][j], galaxy_targets_new['DEC'][j]), file = target_file)

target_file.close()


/home/jail/dlusers/dzaritsky/.cache/ctapipe/cccta-dataserver.in2p3.fr/data/ctapipe-extra/v0.3.4 yale_bright_star_catalog5.fits.gz True
/home/jail/dlusers/dzaritsky/.cache/ctapipe/cccta-dataserver.in2p3.fr/data/ctapipe-extra/v0.3.4 yale_bright_star_catalog5.fits.gz True
/home/jail/dlusers/dzaritsky/.cache/ctapipe/cccta-dataserver.in2p3.fr/data/ctapipe-extra/v0.3.4 yale_bright_star_catalog5.fits.gz True
/home/jail/dlusers/dzaritsky/.cache/ctapipe/cccta-dataserver.in2p3.fr/data/ctapipe-extra/v0.3.4 yale_bright_star_catalog5.fits.gz True
/home/jail/dlusers/dzaritsky/.cache/ctapipe/cccta-dataserver.in2p3.fr/data/ctapipe-extra/v0.3.4 yale_bright_star_catalog5.fits.gz True
/home/jail/dlusers/dzaritsky/.cache/ctapipe/cccta-dataserver.in2p3.fr/data/ctapipe-extra/v0.3.4 yale_bright_star_catalog5.fits.gz True
/home/jail/dlusers/dzaritsky/.cache/ctapipe/cccta-dataserver.in2p3.fr/data/ctapipe-extra/v0.3.4 yale_bright_star_catalog5.fits.gz True
/home/jail/dlusers/dzaritsky/.cache/ctapipe/cccta-datas

From this sample of galaxies we now want to choose the ones that will be accessible for some fraction of the time on nights in September 2023 (at night after astronomical twilight) at the Large Binocular Telescope (LBT) at an arimass < 1.4.

In [6]:
# sets constraints (airmass limit 1.4 and at night).
constraints = [AirmassConstraint(1.4), AtNightConstraint.twilight_nautical()]

# range of dates to span
time_range = Time(["2023-09-01 00:00", "2023-10-01 00:00"])

# parameters of observatory
lbt = Observer.at_site('lbt')

target_coords = SkyCoord(galaxy_targets_new['RA']*u.deg, galaxy_targets_new['DEC']*u.deg, frame='icrs')


table = observability_table(constraints, lbt, target_coords, time_range=time_range)

print('Galaxy and fraction of time satisfying constraints:')

# require observation fraction to be at least 0.1
for i in range(0,len(table)):
    if (table[i][3]>0.10):
        print(galaxy_targets_new["GALAXY"][i], table[i][3])


Galaxy and fraction of time satisfying constraints:
UGC10806 0.10069444444444445
NGC7339 0.2763888888888889


In [7]:
# view the set of galaxies

# subroutine to make the image mosaic
def get_concat(data,rows,galaxy_targets_new,ct,image_number):
    xlen = len(data)
    w = data[0].width
    h = data[0].height
    dst = Image.new('RGB', (3*w + 40 , rows*h+(rows-1)*10),color='white')
    draw = ImageDraw.Draw(dst)
    for i in range(rows):
        for j in range(3):
            count = i*3 + j
            tcount = count + ct*image_number 
            if (count < xlen):
                dst.paste(data[count], (j*w+j*10, i*h+i*10))
                draw.text((j*w+j*10+7,i*h+i*10+15), str(galaxy_targets_new['GALAXY'][tcount]), fill=(255,255,0))
    return dst

#filename = 'targets.jpg'

#try:
#    os.remove(filename)
#except OSError:
#    pass

# maximum images per mosaic
image_number = 100

jlen = 0
x_data = []

xlen = len(galaxy_targets_new)

count = 0
flag = 0

for i in range(xlen):

    if(table[i][3]>0.1):
    
        jlen += 1
        try:
            os.remove('trash.png')
        except:
            pass
        file = open("trash.png", "wb+")
# this sets the link to get the postage stamp image
        link = 'http://legacysurvey.org/viewer/jpeg-cutout/?ra=' + str(galaxy_targets_new['RA'][i]) + '&dec=' + str(galaxy_targets_new['DEC'][i]) +'&layer=dr8&pixscale=5&bands=grz'
# gets the image
        r = requests.get(link)
        if (len(r.content) < 10000):
            r = requests.get(link)
        file.write(r.content)
        file.close()
        
        if (jlen < image_number):
            try:
                x_data.append(Image.open('trash.png'))
            except OSError:
                print('error 1',i)
                jlen -= 1
        else:
            try:
                x_data.append(Image.open('trash.png'))
            except OSError:
                print('error 2')
            jlen = 0
            rows = int(len(x_data)/3) + 1
            filename = 'targets' + str(count) + '.jpg'
            print(filename)
            try:
                os.remove(filename)
            except OSError:
                pass

            get_concat(x_data,rows,gal_targets_new,count,image_number).save(filename)
            count += 1
            x_data = []
    

rows = int(jlen/3) + 1
filename = 'targets' + str(count) + '.jpg'
print(filename)
try:
    os.remove(filename)
except OSError:
    pass

get_concat(x_data,rows,galaxy_targets_new,count,image_number).save(filename)


targets0.jpg


## Conclusion: 


 You can delete SGA-2020.fits, which takes up a lot of disk space.